## Data Science  Amadeus Challenge

05/05/18 & 11/05/18  
Data challenge from the interview process for data scientist in Amadeus

### Exercise 1. Count the number of lines in Python for each file

In [1]:
import pandas as pd
import bz2

We have got 2 compressed bz2 files of 500MB (5GB uncompressed)

_bzip2 is a free and open-source file compression program that uses the Burrows–Wheeler algorithm. It only compresses single files and is not a file archiver_
https://en.wikipedia.org/wiki/Bzip2

In [2]:
path = "/home/dsc/Data/challenge/"
filename_bookings = "bookings.csv.bz2"
filename_searches = "searches.csv.bz2"

1. Attempted `pd.read_csv(path+filename_bookings, compression="bz2")` got error `#ParserError: Error tokenizing data. C error: Expected 1 fields in line 5000009, saw 2`
2. Finally using the bz2 library to open the files without uncompressing them and iterating over each row

In [3]:
%%time
count_lines = 0
with bz2.BZ2File(path+filename_bookings) as fbookings:
    for line in fbookings:
        fbookings
        count_lines += 1
print(count_lines)

10000011
CPU times: user 3min 9s, sys: 1.97 s, total: 3min 11s
Wall time: 3min 11s


In [4]:
%%time
count_lines = 0
with bz2.BZ2File(path+filename_searches) as fsearches:
    for line in fsearches:
        count_lines += 1
print(count_lines)

20390199
CPU times: user 4min 29s, sys: 1.91 s, total: 4min 31s
Wall time: 4min 31s


### Exercise 2. Top 10 arrival airports in the world in 2013

Arrival airport is the column arr_port. It is the IATA code for the airport. To get the total number of passengers for an airport you can sum the column pax grouping by arr_port. Note that there is negative pax that corresponds to cancelations. So to get the total number of passengers that have actually booked you should include the negatives in the sum.
Print the top 10 arrival airports.

Creating a reduced sample file `bookings_sample.csv.bz2` in order to explore the data and devise a strategy to solve the exercise

In [5]:
filename_sample_bookings = "bookings_sample.csv.bz2"

In [6]:
! bzcat {path}{filename_bookings} | head -100000 | bzip2 -c > {path}{filename_sample_bookings} 


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/bookings.csv.bz2, output file = (stdout)


In [7]:
sample_bookings = pd.read_csv(path+filename_sample_bookings,compression="bz2",delimiter="^", usecols=["year", "pax", "arr_port"])

In [8]:
sample_bookings.columns

Index(['arr_port', 'pax', 'year'], dtype='object')

In [9]:
sample_bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 3 columns):
arr_port    99999 non-null object
pax         99999 non-null int64
year        99999 non-null int64
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [10]:
sample_bookings.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


In [11]:
sample_bookings = sample_bookings[sample_bookings["year"] == 2013]

In [13]:
sample_bookings["arr_port"] = sample_bookings["arr_port"].str.strip().str.upper()

In [14]:
sample_top_10 = sample_bookings[["arr_port", "pax"]].groupby(["arr_port"], as_index=False).sum().sort_values(by=["pax"], ascending=False).head(10)
print(sample_top_10)

     arr_port   pax
733       LHR  1006
798       MCO   861
620       JFK   795
706       LAX   761
156       BKK   747
704       LAS   732
1159      SFO   705
957       ORD   686
232       CDG   676
366       DXB   587


1. Ensured the data is only from 2013
2. Ensured there is consistency in the airport codes by trimming white spaces and capitalising all letters
3. Sliced off the necessary columns and grouped by the airport code, then aggregated the result in a sum, sort it and limit it to 10

Now applying this strategy to the whole file.
In order to open the file, used the chunksize argument to iterate through each chunk and appending the result to a new dataframe.

In [15]:
chunks = pd.read_csv(path+filename_bookings,compression="bz2",delimiter="^", usecols=["year", "pax", "arr_port"], chunksize=100000)

In [16]:
top_airports = pd.DataFrame()

In [17]:
%%time
for chunk in chunks:
    chunk = chunk[chunk["year"] == 2013]
    chunk["arr_port"] = chunk["arr_port"].str.strip().str.upper()
    chunk_grouped = chunk[["arr_port", "pax"]].groupby(["arr_port"], as_index=False).sum()
    top_airports = top_airports.append(chunk_grouped)

CPU times: user 3min 51s, sys: 1.57 s, total: 3min 52s
Wall time: 3min 53s


In [19]:
print(top_airports.head())

  arr_port   pax
0      AAE   2.0
1      AAL  16.0
2      AAQ   5.0
3      AAR   4.0
4      ABE  16.0


In [34]:
top_10_airports = top_airports.groupby("arr_port", as_index=False).sum().sort_values(by=["pax"], ascending=False).head(10).reset_index(drop=True)
print(top_10_airports)

  arr_port      pax
0      LHR  88809.0
1      MCO  70930.0
2      LAX  70530.0
3      LAS  69630.0
4      JFK  66270.0
5      CDG  64490.0
6      BKK  59460.0
7      MIA  58150.0
8      SFO  58000.0
9      DXB  55590.0


In [33]:
top_10_airports.to_csv("top_10_airports.csv")

### Bonus exercise 2
Get the name of the city or airport corresponding to that airport (we suggest to have a look at GeoBases in GitHub)

Looking into the GeoBases repository in GitHub found the direct [link](https://raw.githubusercontent.com/opentraveldata/geobases/public/GeoBases/DataSources/Airports/GeoNames/airports_geonames_only_clean.csv) to this table that contains airport codes, and airport names

In [35]:
geobases = pd.read_csv("https://raw.githubusercontent.com/opentraveldata/geobases/public/GeoBases/DataSources/Airports/GeoNames/airports_geonames_only_clean.csv", delimiter="^", usecols=[0,1], header=None, names=["IATA code", "Airport name"])

In [36]:
print(geobases.head())

  IATA code                     Airport name
0       AUH  Abu Dhabi International Airport
1       AZI         Abu Dhabi Bateen Airport
2       AAN     Al Ain International Airport
3       DXB      Dubai International Airport
4       FJR                         Fujairah


In [37]:
top_10_airports = pd.read_csv("top_10_airports.csv", index_col=[0])

In [38]:
print(top_10_airports.head())

  arr_port      pax
0      LHR  88809.0
1      MCO  70930.0
2      LAX  70530.0
3      LAS  69630.0
4      JFK  66270.0


In [39]:
top_10_names = pd.merge(left=top_10_airports,right=geobases, how="left", left_on="arr_port", right_on="IATA code")[["arr_port","pax","Airport name"]]
print(top_10_names)

  arr_port      pax                          Airport name
0      LHR  88809.0               London Heathrow Airport
1      MCO  70930.0         Orlando International Airport
2      LAX  70530.0     Los Angeles International Airport
3      LAS  69630.0        McCarran International Airport
4      JFK  66270.0  John F Kennedy International Airport
5      CDG  64490.0             Paris - Charles-de-Gaulle
6      BKK  59460.0                          Suvarnabhumi
7      MIA  58150.0           Miami International Airport
8      SFO  58000.0   San Francisco International Airport
9      DXB  55590.0           Dubai International Airport


### Exercise 3. Plot the monthly number of searches for flights arriving in Malaga, Madrid or Barcelona

Creating a sample file `searches_sample.csv.bz2`

In [40]:
filename_sample_searches = "searches_sample.csv.bz2"

In [41]:
! bzcat {path}{filename_searches} | head -100000 | bzip2 -c > {path}{filename_sample_searches} 


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/searches.csv.bz2, output file = (stdout)


In [42]:
sample_searches = pd.read_csv(path+filename_sample_bookings,compression="bz2",delimiter="^")

In [43]:
sample_searches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 38 columns):
act_date               99999 non-null object
source                 99999 non-null object
pos_ctry               99999 non-null object
pos_iata               99999 non-null object
pos_oid                99999 non-null object
rloc                   99999 non-null object
cre_date               99999 non-null object
duration               99999 non-null int64
distance               99999 non-null int64
dep_port               99999 non-null object
dep_city               99999 non-null object
dep_ctry               99999 non-null object
arr_port               99999 non-null object
arr_city               99999 non-null object
arr_ctry               99999 non-null object
lst_port               99999 non-null object
lst_city               99999 non-null object
lst_ctry               99999 non-null object
brd_port               99999 non-null object
brd_city               99999 non-null 

In [44]:
sample_searches.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [45]:
sample_searches.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [46]:
sample_searches.columns = sample_searches.columns.str.strip()

# TBC